# Welcome to TapeAgents!

**TapeAgents** is a framework to build, debug, serve and optimize your AI agent. It takes a holistic view of the agent lifecycle and aims to support you at all stages. The main distinguishing feature of the framework is that by design a **TapeAgent** creates its  
**Tape**: a compherensive semantic log of the agent's session that greatly facilitates audit, debugging, finetuning, agent optimization, etc.

In this tutorial you will learn:
- how to create TapeAgents using the low-level API
- run and resume TapeAgents
- have one TapeAgent reuse another TapeAgent's tape as training data

In upcoming versions of this tutorial you will also learn: 
- how to make a team TapeAgent with subagents
- how to build TapeAgents using available high-level APIs
- how to build a TapeAgent that streams partial steps

Other tutorials and examples will cover:
- code execution and browser use
- finetuning
- the TapeAgents apps (Studio and Browser)

# 0. Setup

You have outputs in all cells in this notebook, so you can just scroll through! If you want to run it, here are the steps.

#### 0. Install conda
https://conda.io/projects/conda/en/latest/user-guide/install/index.html

#### 1. All commands assume execution from the tapeagents directory.
```bash
cd tapeagents/
```

#### 2. Create conda environment and install `tapeagents` and its dependencies:
```bash
make setup
```

#### 3. Set Jupyter Notebook kernel to your newly created `tapeagents` conda environment
https://code.visualstudio.com/docs/datascience/jupyter-notebooks#_create-or-open-a-jupyter-notebook

#### 4. Set your LLM API keys and other minor things

In [1]:
import os

# os.environ["OPENAI_API_KEY"] = "" # put your https://platform.openai.com/ key here
# os.environ["OPENAI_ORGANIZATION"] = "" # optional if you use your personal key
# os.environ["TOGETHER_API_KEY"] = "" # put your https://together.ai/ key here
# os.environ["TAPEAGENTS_LLM_TOKEN"] = os.environ["TOGETHER_API_KEY"] # Use Together API Key for llms.LLAMA
# os.environ["HF_TOKEN"] = "" # put your huggingface token here

os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
today = "2024-09-17" # fixed date for reproducible tests

In [2]:
# Login to Hugging Face Hub
if "PYTEST_CURRENT_TEST" not in os.environ: # skip login during tests
    from huggingface_hub import login
    login(token=os.environ.get("HF_TOKEN"))

#### 5. Access Hugging Face Gated Models
Make sure you have access to each model we are going to use (read more [here](https://huggingface.co/docs/hub/en/models-gated#access-gated-models-as-a-user)):
- https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct

##### Troubleshoot
- If you receive a 401 error, it means the authentication failed.
- If you receive a 403 error, it means you are authenticated but not authorized to access the specific model.


# 1. Your first TapeAgent

In this section we will build the simplest possible "hello world" agent. We will then go through all the new concepts that you need to know to understand the code. This section is quite long, but with the solid foundation you acquire here other TapeAgent tutorials will be easy to process.

Without further ado, here's the code!

In [3]:
from tapeagents.agent import Agent, Node
from tapeagents.core import SetNextNode, Prompt
from tapeagents.dialog_tape import AssistantStep, DialogTape, UserStep
from tapeagents.llms import LLMStream, LiteLLM
from tapeagents.prompting import tape_to_messages

llm = LiteLLM(model_name="gpt-4o-mini-2024-07-18")


def make_prompt(agent: Agent, tape: DialogTape) -> Prompt:
    return Prompt(messages=tape_to_messages(tape))


def generate_steps(agent: Agent, tape: DialogTape, llm_stream: LLMStream):
    """
    llm_stream contains the text of the LLM model response
    """
    yield AssistantStep(content=llm_stream.get_text())
    yield SetNextNode(next_node=0)  # Which node to execute next, more on that later


node = Node().with_prompt(make_prompt).with_generate_steps(generate_steps)
nodes = [node]
agent = Agent[DialogTape].create(llm, nodes=nodes)
start_tape = DialogTape(steps=[UserStep(content="Tell me about Vulcan in 3 sentences")])
final_tape = agent.run(start_tape).get_final_tape()
print(final_tape.model_dump_json(indent=2))

{
  "metadata": {
    "id": "7477a038-46fc-4fd5-9bf2-465d03cb6e24",
    "parent_id": "99db349b-8fd1-4e59-bb5d-caeb13135f8d",
    "author": "",
    "author_tape_id": null,
    "n_added_steps": 2,
    "error": null,
    "result": null
  },
  "context": null,
  "steps": [
    {
      "metadata": {
        "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
        "prompt_id": "",
        "node": "",
        "agent": "",
        "other": {}
      },
      "content": "Tell me about Vulcan in 3 sentences",
      "kind": "user"
    },
    {
      "metadata": {
        "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
        "prompt_id": "3b2f7f58-a41e-4b6d-b6c4-b960b2159f59",
        "node": "",
        "agent": "Agent",
        "other": {}
      },
      "content": "Vulcan is a fictional planet in the \"Star Trek\" universe, primarily known as the homeworld of the Vulcan species, including the iconic character Spock. The planet is characterized by its harsh desert environment, advanced technology, a

Now let's learn about tapes, steps, prompts, llm streams, nodes and agents.

### Tape

The fundamental concept of the TapeAgents is the `Tape`, a comprehensive semantic level log of the agent's session. A `Tape` contains a context and a sequence of `Step` objects. As you can see, a TapeAgent runs by adding steps (such as `UserStep` or `AssistantStep`) to the _tape_. This example uses the `DialogTape` tape, which is a basic tape for user-assistant conversations. Let's see what are the possible steps in a `DialogTape`.

In [4]:
# We use Python generics to instantiate many different Tape types by
# specifying different Context and Step types. In the output of this cell,
# look at Union[UserStep, AssistantStep, ...]
# for the list of possible step types in the DialogTape.
DialogTape


tapeagents.core.Tape[Union[DialogContext, NoneType], Union[UserStep, ToolResult, SystemStep, AssistantThought, SetNextNode, Pass, Call, Respond, FinalStep, AssistantStep, ToolCalls]]

Some of these steps should be familiar to you. `UserStep`, `AssistantStep`, `SystemStep` and `ToolResult` correspond to `role=user`, `role=assistant`, `role=system` and `role=tool` LLM API messages respectively. `ToolCalls` and `AssistantThought` correspond to assistant messages where the LLM requests a tool call or produces an intermediate thought that is not meant to be shown to the user. `SetNextNode` and `Pass` are TapeAgent's internal step to control which node it should run at the next iteration (more on this below).

### Prompt format; LLMs

We use the industry-standard "chat.completions" prompt format in TapeAgents: a list of user/assistant/system/tool messages plus tool schemas.

In [5]:
# Almost all classes in TapeAgents are Pydantic base models.
# This allows easy validation, serialization and instrospection. For example,
# here we are able to list all the fields in the Prompt model.
Prompt.model_fields


{'id': FieldInfo(annotation=str, required=False, default_factory=<lambda>),
 'tools': FieldInfo(annotation=Union[list[dict], NoneType], required=False, default=None),
 'messages': FieldInfo(annotation=list[dict], required=False, default=[])}

The LLMs in TapeAgent take `Prompt` and return an `LLMStream` object. The `LLMStream` object can be used both to fast-forward to the complete response text and to stream partial outputs step by step. 

In [6]:
from tapeagents.llms import LLAMA

llama = LLAMA(
    base_url="https://api.together.xyz",
    model_name="meta-llama/Meta-Llama-3-70B-Instruct-Turbo",
    tokenizer_name="meta-llama/Meta-Llama-3-70B-Instruct",
    # you must use stream=True if you wish to have message chunks in your LLM events
    stream=True,
)
# Streaming
prompt = Prompt(messages=[{"role": "user", "content": "Write hello world in Java"}])
for event in llama.generate(prompt):
    print(event.chunk, end="")
# No streaming
# (note: you can not use Prompt object for more than 1 LLM call in TapeAgents)
prompt = Prompt(messages=[{"role": "user", "content": "Write hello world in C"}])
print("\n" + "-" * 30)
print(llama.generate(prompt).get_text())


Here is the classic "Hello, World!" program

 in Java:
```
public class HelloWorld {
  public static void main(String[] args) {
    System.out.println("Hello, World!");


  }
}
```
Let me explain what's going on:

* `public class HelloWorld {` declares

 a new Java class called `HelloWorld`. The `public` access modifier means

 that the class can be accessed from outside the package where it's defined.
* `public static void main(String

...

 compile it using the `javac` command, and finally run it using the `java` command. Here

 are the steps:

1. Save the code to a file called `HelloWorld.java`.
2. Open a terminal or

 command prompt and navigate to the directory where you saved the file.
3. Compile the program using the following command: `javac HelloWorld.java`


4. Run the program using the following command: `java HelloWorld`

This should print "Hello, World!" to the console!None
------------------------------


Here is the classic "Hello, World!" program in C:
```
#include <stdio.h>

int main() {
    printf("Hello, World!\n");
    return 0;
}
```
Let me explain what each line does:

1. `#include <stdio.h>`: This line tells the compiler to include the `stdio.h` header file, which provides input/output functions such as `printf`.
2. `int main()`: This line declares a function called `main`, which is the entry point of the program. The `int` keyword specifies that the function returns an integer value.
3. `printf("Hello, World!\n");`: This line uses the `printf` function to print the string "Hello, World!" followed by a newline character (`\n`) to the console.
4. `return 0;`: This line returns an integer value of 0 to indicate that the program has completed successfully.

To compile and run this program, you can use a C compiler such as GCC:
```
$ gcc hello.c -o hello
$ ./hello
Hello, World!
```
This will compile the `hello.c` file and create an executable file called `hello`, which you can then

In the example above we use the easiest way to create a prompt from the tapes: `tape_to_messages`. Under the hood this method uses `step.llm_dict()` method of all non-control steps in the tape to create the prompt:

In [7]:
print((user := UserStep(content="hi AI!")).llm_dict())
print((assistant := AssistantStep(content="hello human")).llm_dict())
print(tape_to_messages(DialogTape(steps=[user, assistant])))


{'content': 'hi AI!', 'kind': 'user'}
{'content': 'hello human', 'kind': 'assistant'}
[{'role': 'user', 'content': 'hi AI!'}, {'role': 'assistant', 'content': 'hello human'}]


A key priority in TapeAgents is making use of the data that running the agent generates. To make this possible, some TapeAgent LLMs know how to make their finetuning data:

In [8]:
from tapeagents.core import LLMOutput


simple_tape = DialogTape(
    steps=[
        UserStep(content="Say bla 3 times and foo 2 times"),
        AssistantStep(content="Sure! Let me say bla bla bla foo foo"),
    ]
)
prompt = Prompt(messages=tape_to_messages(simple_tape[:1]))
output = agent.make_llm_output(simple_tape, index=1)
text = llama.make_training_text(prompt=prompt, output=output)
print("--- ALL TEXT ---")
print(text.text)
print("--- PREDICTED CHARACTERS ---")
print(text.output_text)


--- ALL TEXT ---
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Say bla 3 times and foo 2 times<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Sure! Let me say bla bla bla foo foo<|eot_id|>
--- PREDICTED CHARACTERS ---
<|start_header_id|>assistant<|end_header_id|>

Sure! Let me say bla bla bla foo foo<|eot_id|>


### Node

A node represents an uninterruptible atom of TapeAgent's computation. When TapeAgents runs a node, it uses its two main functions: `make_prompt` to create LLM Prompt from the tape and `generate_steps` to create new steps from the LLM output. To build a node, you can replace the default implementations of these functions using `with_prompt` and `with_generate_steps` methods. Or you can subclass `Node` and override these functions. Note that `generate_steps` must be a generator, a design choice we made to make TapeAgents a streaming-friendly framework. 

Let's see what the node from the above example can do.

In [9]:
from tapeagents.llms import LLMEvent


def make_prompt(agent, tape: DialogTape) -> Prompt:
    return Prompt(messages=tape_to_messages(tape))


def generate_steps(agent, tape, llm_stream: LLMStream):
    yield AssistantStep(content=llm_stream.get_text())
    yield SetNextNode(next_node=0)  # Continue to the same first node


node = Node().with_prompt(make_prompt).with_generate_steps(generate_steps)

# Let's run "make_prompt" in isolation.
prompt = node.make_prompt(agent=None, tape=DialogTape(steps=[UserStep(content="Hi, AI!")]))
print(f"Raw node prompt:\n{prompt}\n")


# Now, let's run "generate_steps" in isolation.
# We need to construct a fake LLMStream to do that.
def _generator():
    yield LLMEvent(output=LLMOutput(content="Hello, human!"))


stream = LLMStream(_generator(), Prompt())
step_stream = node.generate_steps(agent=None, tape=DialogTape(), llm_stream=stream)
print(f"Steps produced by node's generator:\n{list(step_stream)}\n")

# When the agent runs a node, it is a equivalent to the following three steps:
# Step 1: make a prompt
start_tape = DialogTape(steps=[UserStep(content="Hi, AI!")])
prompt = node.make_prompt(agent, start_tape)
# Step 2: construct the LLMStream from the prompt (happens inside the agent)
stream = llama.generate(prompt)
# Step 3: generate steps that the agent will then add to the tape
print("Produced Steps:")
for step in node.generate_steps(agent, start_tape, stream):
    print(step.model_dump_json(indent=2))


Raw node prompt:
id='028e3269-3d07-4725-970d-938535f5e009' tools=None messages=[{'role': 'user', 'content': 'Hi, AI!'}]

Steps produced by node's generator:
[AssistantStep(metadata=StepMetadata(id='dc2542d8-cacb-4125-9e42-cc3b42b9a3f8', prompt_id='', node='', agent='', other={}), content='Hello, human!', kind='assistant'), SetNextNode(metadata=StepMetadata(id='dc2542d8-cacb-4125-9e42-cc3b42b9a3f8', prompt_id='', node='', agent='', other={}), kind='set_next_node', next_node=0)]

Produced Steps:


{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "",
    "node": "",
    "agent": "",
    "other": {}
  },
  "content": "Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat?",
  "kind": "assistant"
}
{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "",
    "node": "",
    "agent": "",
    "other": {}
  },
  "kind": "set_next_node",
  "next_node": 0
}


### Agent and its nodes

The TapeAgent agent iteratively runs the nodes and appends the steps generated by each node to the tape. To select which next node to run, internally a TapeAgent computes the **tape view** object. The Tape remains the only **state** that the agent uses, the view only represents its content in a way that is convenient for the agent to use.


In [10]:
from tapeagents.view import TapeViewStack

# The "top" view in the tape view stack is the view of current agent. Initially `top.next_node` is 0".
tape1 = DialogTape(steps=[UserStep(content="Hi, AI!")])
print(TapeViewStack.compute(tape1).top.next_node)

# When the agent computes the view, it bumps up `top.next_node` every time it encounters a step with a new `prompt_id``.
# The new prompt_id on the tape signals to the agent the current node has run.
tape2 = DialogTape(steps=[UserStep(content="Hi, AI!"), AssistantStep(prompt_id="123", content="AI here, how I can help?")])
print(TapeViewStack.compute(tape2).top.next_node)

# The SetNextNode step on the tape changes `top.next_node` to the value of the `next_node` field in the SetNextNode step.
tape3 = DialogTape(
    steps=[
        UserStep(content="Hi, AI!"),
        AssistantStep(prompt_id="123", content="AI here, how I can help?"),
        SetNextNode(next_node=0),
    ]
)
print(TapeViewStack.compute(tape3).top.next_node)


0
0
0


By default the agent stops after the last node has produced an `Action` step. The action steps are the step by which the agent requests information from the environment. For example, `AssistantStep` is an `Action` as it indicates the agent awaits the user response, `ToolCalls` is an action requesting tool call results. Let's look at all possible steps in `DialogTape` tape and see which of them are actions, observations and thoughts.

In [11]:
from tapeagents.core import Action, Pass, Thought, Observation
from tapeagents.dialog_tape import AssistantThought, ToolCalls, ToolResult

assert all([issubclass(step_class, Action) for step_class in [AssistantStep, ToolCalls]])
assert all([issubclass(step_class, Thought) for step_class in [AssistantThought, SetNextNode, Pass]])
assert all([issubclass(step_class, Observation) for step_class in [UserStep, ToolResult]])


Now we are ready to look at a simplified summary of the corner-stone `agent.run` algorithm.

1. Compute the new tape view
2. Choose the active agent (more on multi-agent TapeAgents later)
3. Choose the active node
4. Run the node and add steps on the tape
5. If the last node yielded an action, then stop, else repeat.

`agent.run` returns an `AgentStream` object which allows iterating through the agent's steps (or partial steps when streaming) and fast-forwardin to the complete new tape with `get_final_tape`.

#### Converse with a TapeAgent

Lets continue conversation with the agent that previously responded to us with the Vulcan definition from the StarTrek.
Remember, the session stored in the tape **final_tape**.

In [12]:
tape_to_continue = final_tape + [UserStep(content="No, I mean Vulcan the company")]
continued_tape = agent.run(tape_to_continue).get_final_tape()
print(continued_tape.model_dump_json(indent=2))


{
  "metadata": {
    "id": "6f112856-9ae1-466a-98e7-e51ca3819177",
    "parent_id": "c440926d-3bbe-4b4d-a870-8a4642d90b75",
    "author": "",
    "author_tape_id": null,
    "n_added_steps": 2,
    "error": null,
    "result": null
  },
  "context": null,
  "steps": [
    {
      "metadata": {
        "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
        "prompt_id": "",
        "node": "",
        "agent": "",
        "other": {}
      },
      "content": "Tell me about Vulcan in 3 sentences",
      "kind": "user"
    },
    {
      "metadata": {
        "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
        "prompt_id": "3b2f7f58-a41e-4b6d-b6c4-b960b2159f59",
        "node": "",
        "agent": "Agent",
        "other": {}
      },
      "content": "Vulcan is a fictional planet in the \"Star Trek\" universe, primarily known as the homeworld of the Vulcan species, including the iconic character Spock. The planet is characterized by its harsh desert environment, advanced technology, a

Note, that the agent is able to continue talking to you thanks for `SetNextNode(next_node=0)` step that `generate_steps` produced. If you try to remove this step as an exercise, the agent will crash because there is only one node.

#### Tape rendering

LLM agents create a lot of data that can be overwhelming to process. In TapeAgents we render the tape with the associated prompts and outputs into a more readable HTML for you. To make this work, we store prompts and outputs in an SQLite database every time you call `agent.run()`.

Here's how to use tape rendering in the notebook:

In [13]:
from tapeagents.rendering import PrettyRenderer, render_tape_with_prompts
from IPython.display import HTML

HTML(render_tape_with_prompts(continued_tape, PrettyRenderer()))


# 2. Your TapeAgent with planning and tools

Let's build a TapeAgent that plans and acts. We will be using OpenAI function calling capabilities in this example.

In [14]:
from tapeagents.core import SetNextNode
from tapeagents.dialog_tape import AssistantThought, ToolCalls
from tapeagents.environment import ToolEnvironment
from tapeagents.runtime import main_loop
from examples.intro_tools import get_stock_ticker, get_stock_data

system_instruction = f"""
You will help the user to learn about financials of companies.
Use as many relevant tools as possible to include more details and facts in your responses.
Today is {today}.
"""
system_message = {"role": "system", "content": system_instruction}

env = ToolEnvironment([get_stock_ticker, get_stock_data])


def make_planning_prompt(agent, tape: DialogTape) -> Prompt:
    guidance = "Write a natural language plan on how to use tools help the user. Output a list of numbered items, like 1., 2., 3., etc."
    guidance_message = {"role": "user", "content": guidance}
    return Prompt(
        messages=[system_message] + tape_to_messages(tape) + [guidance_message], tools=env.get_tool_schema_dicts()
    )


def generate_planning_steps(agent, tape, llm_stream: LLMStream):
    if content := llm_stream.get_message().content:
        yield AssistantThought(content=content)
    else:
        raise ValueError()


def make_prompt(agent, tape: DialogTape) -> Prompt:
    guidance = "Follow the plan you created to earlier. When you are done, respond to the user."
    guidance_message = {"role": "user", "content": guidance}
    return Prompt(
        messages=[system_message] + tape_to_messages(tape) + [guidance_message], tools=env.get_tool_schema_dicts()
    )


def generate_steps(agent, tape, llm_stream: LLMStream):
    m = llm_stream.get_message()
    if m.content:
        yield AssistantStep(content=m.content)
        yield SetNextNode(next_node=0)
    elif m.tool_calls:
        yield ToolCalls(tool_calls=m.tool_calls)
        yield SetNextNode(next_node=1)
    else:
        raise ValueError()


agent1 = Agent.create(
    LiteLLM(model_name="gpt-4o", parameters={"temperature": 0.1}),
    nodes=[
        Node().with_prompt(make_planning_prompt).with_generate_steps(generate_planning_steps),
        Node().with_prompt(make_prompt).with_generate_steps(generate_steps),
    ],
)

print("Run the agent!")
final_tape1 = None
for event in main_loop(agent1, DialogTape() + [UserStep(content="Tell me about Vulcan in 3 sentences")], env):
    if ae := event.agent_event:
        if ae.step:
            print(ae.step.model_dump_json(indent=2))
        if ae.final_tape:
            final_tape1 = ae.final_tape
    if event.observation:
        print(event.observation.model_dump_json(indent=2))
assert final_tape1
print("Final tape:")
HTML(render_tape_with_prompts(final_tape1, PrettyRenderer()))


Run the agent!


{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "a1023fa2-8d0b-4790-84f9-adc966831f1a",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "content": "1. Use the `functions.get_stock_ticker` tool to obtain the stock ticker symbol for Vulcan.\n2. Use the `functions.get_stock_data` tool to retrieve the stock prices for Vulcan over a specified date range.\n3. Summarize the information obtained from the stock ticker and stock data to provide a comprehensive overview of Vulcan's financials.",
  "kind": "assistant_thought"
}


{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "4c715e1d-071c-402c-86d4-fa2158746fee",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "tool_calls": [
    {
      "function": {
        "arguments": "{\"company_name\":\"Vulcan\"}",
        "name": "get_stock_ticker"
      },
      "id": "call_S2Il5yCMNsQZyBgensfmr7dh",
      "type": "function"
    }
  ],
  "kind": "assistant"
}
{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "4c715e1d-071c-402c-86d4-fa2158746fee",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "kind": "set_next_node",
  "next_node": 1
}


{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "",
    "node": "",
    "agent": "",
    "other": {}
  },
  "content": "VMC",
  "tool_call_id": "call_S2Il5yCMNsQZyBgensfmr7dh",
  "kind": "tool"
}


{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "9ff16eb9-0fd5-4a6c-936a-4fce4511a201",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "tool_calls": [
    {
      "function": {
        "arguments": "{\"symbol\": \"VMC\", \"start_date\": \"2024-01-01\", \"end_date\": \"2024-09-16\"}",
        "name": "get_stock_data"
      },
      "id": "call_zXXvyYM7xrC9MDgaH44GdCbb",
      "type": "function"
    }
  ],
  "kind": "assistant"
}
{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "9ff16eb9-0fd5-4a6c-936a-4fce4511a201",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "kind": "set_next_node",
  "next_node": 1
}
{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "",
    "node": "",
    "agent": "",
    "other": {}
  },
  "content": "[('2024-01-02', 223.60000610351562), ('2024-01-04', 220.67999267578125), ('2024-01-08', 224.0), ('2024-01-10', 226.11000061035156)

{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "d021fdef-b8fa-4431-be17-e36cdd5d22b6",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "content": "Vulcan Materials Company, trading under the stock ticker symbol VMC, is a major producer of construction aggregates, primarily crushed stone, sand, and gravel. Over the course of 2024, Vulcan's stock price has shown significant fluctuations, starting at approximately $223.60 in early January and reaching highs of around $274.36 in late March, before experiencing a decline to around $237.47 by mid-September. This volatility reflects the dynamic nature of the construction materials market and the broader economic conditions impacting the industry.",
  "kind": "assistant"
}
{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "d021fdef-b8fa-4431-be17-e36cdd5d22b6",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "kind": "set_next_node",
  "next_node"

The main new thing in this example is the environment. In TapeAgents framework the environment responds to the agent `Action` steps with `Observation` steps. We expect you to use the environment to encapsulate tool use, retrieval, code execution: everything that is non-deterministic, non-stationary, or computationally heavy. On the contrary, we encourage you to implements the agent's deterministic decision-making in `make_prompt` and `generate_steps` methods.

Here we use a pre-defined `main_loop` orchestrator to run the agent and the environment. `main_loop` is a generator of events that you can use as you wish. You are free to implement your own orchestration paradigm with a fine-grained control over what actions get to be executed.

# 3. Agent configuration, resumption

Let's try building a similar agent with an open-weights LLAMA3.1 70B models. Conveniently, [Together AI](together.ai) offers API endpoints. You can create an account and get API key with some free quota.

We've found that LLAMA3 function-calling is not yet battle-ready. We will use the structured output approach to make it call tools instead. We are also making this agent trainable by adding `make_llm_output` methods to each node. `Node.make_llm_output` defines how a node can reconstruct the LLM completion message that would be required to make the steps from the given tape at the given index. You can think of `Node.make_llm_output` as the inverse of `Node.generate_steps`.

When you run the code below, you might see a different behavior every time. Often the LLAMA-based agent gets stuck in a loop. We will look into how TapeAgents supports you in addressing this issue by
- tuning the prompt and resuming the agent exactly where it got stuck 
- producing training text from a different agent's tape 

In [15]:
import json
from tapeagents.core import LLMOutput
from tapeagents.llms import LLAMA
from litellm.utils import ChatCompletionMessageToolCall
from litellm.utils import Function

from tapeagents.prompting import step_to_message

env = ToolEnvironment([get_stock_ticker, get_stock_data])

system_instruction = f"""
You will help the user to learn about financials of companies.
Use as many relevant tools as possible to include more details and facts in your responses.
Today is {today}.

You have access to the following tools: {env.get_tool_schema_dicts()}"""

planning_guidance = "Write a natural language plan on how to use tools help the user. Output a list of numbered items, like 1., 2., 3., etc."

call_or_respond_guidance = """
Follow the plan you created earlier. When you are done, respond to the user.
If you want to call a or several tools, output JSON like this
{"kind": "tool_call", "tool_name": "...", "parameters": "... unquoted parameters json ..."}
If you have called all the tools in the plan, respond to the user with the JSON of the form
{"kind": "response", "content": "... you response ... "}.
Output ONE JSON OBJECT ONLY PER LINE ONLY AND NOTHING ELSE.
"""


def make_planning_prompt(agent, tape: DialogTape) -> Prompt:
    system_message = {"role": "system", "content": system_instruction}
    guidance_message = {"role": "user", "content": agent.templates["planning"]}
    return Prompt(messages=[system_message] + tape_to_messages(tape) + [guidance_message])


def generate_planning_steps(agent, tape, llm_stream: LLMStream):
    if content := getattr(llm_stream.get_message(), "content", None):
        yield AssistantThought(content=content)
    else:
        raise ValueError()


def make_planning_llm_output(agent, tape: DialogTape, index: int) -> LLMOutput:
    if not isinstance(current := tape[index], AssistantThought):
        raise ValueError()
    return LLMOutput(role="assistant", content=current.content)


def _llm_message_content(step: AssistantStep | ToolCalls):
    """Helper function to make both the prompt and the target completion"""
    match step:
        case AssistantStep():
            return json.dumps({"kind": "response", "content": step.content})
        case ToolCalls():
            content = ""
            for tool_call in step.tool_calls:
                if content:
                    content += "\n"
                content += json.dumps(
                    {
                        "kind": "tool_call",
                        "tool_name": tool_call.function.name,
                        "parameters": json.loads(tool_call.function.arguments),
                    }
                )
            return content
        case _:
            raise ValueError()


def make_prompt(agent, tape: DialogTape) -> Prompt:
    system_message = {"role": "system", "content": system_instruction}
    guidance_message = {"role": "user", "content": agent.templates["call_or_respond"]}
    messages = [system_message]
    for step in tape:
        if isinstance(step, (ToolCalls)):
            messages.append({"role": "assistant", "content": _llm_message_content(step)})
        elif not isinstance(step, SetNextNode):
            messages.append(step_to_message(step))
    messages += [guidance_message]
    return Prompt(messages=messages)


def generate_steps(agent, tape, llm_stream: LLMStream):
    m = llm_stream.get_message()
    try:
        assert m.content
        tool_calls = []
        response = None
        for line in m.content.split("\n"):
            data = json.loads(line)
            if data.get("kind") == "response":
                response = data["content"]
            elif data.get("kind") == "tool_call":
                tool_call = ChatCompletionMessageToolCall(
                    function=Function(name=data["tool_name"], arguments=json.dumps(data["parameters"])),
                    # tool call must be a unique string, it helps to make it something deterministic
                    id=f"tool_call_{len(tool_calls)}_node_starts_at_{len(tape)}",
                )
                tool_calls.append(tool_call)
            else:
                yield AssistantStep(content="Invalid LLM output: kind field must be 'response' or 'tool_call'")
        if response and tool_calls:
            yield AssistantStep(content="Invalid LLM output: response and tool_call cannot be in the same message")
        if response:
            yield AssistantStep(content=response)
            yield SetNextNode(next_node=0)
        if tool_calls:
            yield ToolCalls(tool_calls=tool_calls)
            yield SetNextNode(next_node=1)
    except Exception as e:
        yield AssistantStep(content="Invalid JSON object: " + str(e))


def make_llm_output(agent, tape: DialogTape, index: int) -> LLMOutput:
    if not isinstance(step := tape[index], AssistantStep | ToolCalls):
        raise ValueError()
    content = _llm_message_content(step)
    return LLMOutput(role="assistant", content=content)


agent2 = Agent.create(
    LLAMA(
        base_url="https://api.together.xyz",
        model_name="meta-llama/Meta-Llama-3-70B-Instruct-Turbo",
        tokenizer_name="meta-llama/Meta-Llama-3-70B-Instruct",
        parameters=dict(temperature=0.01),
    ),
    templates={
        "system": system_instruction,
        "planning": planning_guidance,
        "call_or_respond": call_or_respond_guidance,
    },
    nodes=[
        Node()
        .with_prompt(make_planning_prompt)
        .with_generate_steps(generate_planning_steps)
        .with_llm_output(make_planning_llm_output),
        Node().with_prompt(make_prompt).with_generate_steps(generate_steps).with_llm_output(make_llm_output),
    ],
)

final_tape2 = None
print("Run LLAMA agent!")
for event in main_loop(
    agent2, DialogTape() + [UserStep(content="Tell me about Vulcan Materials in 3 sentences")], env, max_loops=3
):
    if ae := event.agent_event:
        if ae.step:
            print(ae.step.model_dump_json(indent=2))
        if ae.final_tape:
            final_tape2 = ae.final_tape
    if event.observation:
        print(event.observation.model_dump_json(indent=2))
assert final_tape2 is not None
print("Final tape:")
HTML(render_tape_with_prompts(final_tape2, PrettyRenderer()))


Run LLAMA agent!


{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "866fda12-27ba-4aa1-ac3c-efe14dafd9bc",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "content": "Here's a plan to help the user learn about Vulcan Materials using the provided tools:\n\n1. First, use the `get_stock_ticker` function to retrieve the stock ticker symbol for Vulcan Materials by passing \"Vulcan Materials\" as the `company_name` parameter. This will give us the stock ticker symbol, which we'll use in the next step.\n\n2. Once we have the stock ticker symbol, use the `get_stock_data` function to retrieve the historical stock prices for Vulcan Materials. We can pass the stock ticker symbol, a start date (e.g., 2020-01-01), and an end date (e.g., 2024-09-17) as parameters to get the stock prices for the specified date range.\n\n3. Analyze the retrieved stock prices to provide an overview of Vulcan Materials' stock performance over the specified period. This could include calcul

{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "3898aee1-20fe-462e-81e2-8642b10978d7",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "tool_calls": [
    {
      "function": {
        "arguments": "{\"company_name\": \"Vulcan Materials\"}",
        "name": "get_stock_ticker"
      },
      "id": "tool_call_0_node_starts_at_2",
      "type": "function"
    }
  ],
  "kind": "assistant"
}
{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "3898aee1-20fe-462e-81e2-8642b10978d7",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "kind": "set_next_node",
  "next_node": 1
}
{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "",
    "node": "",
    "agent": "",
    "other": {}
  },
  "content": "VMC",
  "tool_call_id": "tool_call_0_node_starts_at_2",
  "kind": "tool"
}


{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "737fc856-3b7c-4905-a11d-a62a95e21e85",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "tool_calls": [
    {
      "function": {
        "arguments": "{\"symbol\": \"VMC\", \"start_date\": \"2020-01-01\", \"end_date\": \"2024-09-17\"}",
        "name": "get_stock_data"
      },
      "id": "tool_call_0_node_starts_at_5",
      "type": "function"
    }
  ],
  "kind": "assistant"
}
{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "737fc856-3b7c-4905-a11d-a62a95e21e85",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "kind": "set_next_node",
  "next_node": 1
}
{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "",
    "node": "",
    "agent": "",
    "other": {}
  },
  "content": "[('2020-01-02', 142.72000122070312), ('2020-01-27', 138.8800048828125), ('2020-02-19', 135.39999389648438), ('2020-03-12', 107.1200

{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "54a19edf-9af3-4de3-be13-b0c1dde2c155",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "content": "Vulcan Materials is a leading producer of construction materials, including aggregates, asphalt, and ready-mixed concrete. Its stock, VMC, has shown a significant increase in value over the past 4 years, with a highest price of around 273.73 on 2024-03-14 and a lowest price of around 99.21 on 2020-04-03. The company's stock has been trending upward, with some fluctuations, indicating a strong performance in the construction materials industry.",
  "kind": "assistant"
}
{
  "metadata": {
    "id": "dc2542d8-cacb-4125-9e42-cc3b42b9a3f8",
    "prompt_id": "54a19edf-9af3-4de3-be13-b0c1dde2c155",
    "node": "",
    "agent": "Agent",
    "other": {}
  },
  "kind": "set_next_node",
  "next_node": 0
}
Final tape:


Sometimes the above agent works well, but quite likely you are seeing that the LLAMA-based agent is having trouble. Let's try to help it. For reproducibility, we'll use a pre-recorded failed tape.

In [16]:
with open("assets/failed_tape.json") as src:
    failed_tape = DialogTape.model_validate(json.load(src))
agent2b = agent2.model_copy(deep=True)
agent2b.templates["call_or_respond"] += (
    "REMEMBER: check what tool calls you have already made. Do not do the same call again!"
)
resume_from_step8 = agent2b.run(failed_tape[:8]).get_final_tape()
HTML(render_tape_with_prompts(resume_from_step8, PrettyRenderer()))


We found that this helpful hint often gets LLAMA-based agent unstuck. Note how easy it was to test it thanks to the ability of the agent to resume from step 8!

# 4. Tape reuse and training data

Another way to help this agent (or one with an even smaller LLM) is to finetune the LLM. And the most important step towards finetuning is making the training data!

There are two ways to make training data in TapeAgents:
- the basic one: use the `LLMCall` structures that the agent created when it produced the tape. You can retrieve them from the SQLite storage and convert into training text.
- the much powerful one: call `agent.reuse` to reconstructed the prompts and outputs **and** to validate that with the reconstructed LLMCalls the agent would indeed create the given tape

The big advantage of the second approach is that it allows you to use the tape from a teacher agent (think slower and more expensive) to train a student agent (think faster and cheaper). Or to train an agent on its own revised tapes.

Of course, restrictions apply: the tape by agent A may not be reusable by agent B directly. You might have to add/remove some steps. But at least `agent.reuse` verifies if your tape modifications led to creation of a tape that the agent B can indeed produce.

#### Make training data from the past LLM calls

In [17]:
from tapeagents.observe import retrieve_tape_llm_calls

llm_calls = retrieve_tape_llm_calls(final_tape2)
print(f"Retrieved {len(llm_calls)} LLM calls from the tape.")
# under the hood agent2 will route this request to its llm
example_text = agent2.make_training_text(list(llm_calls.values())[0])
print("From the first retrieved LLM call, the LLM will be trained to predict this text:")
print("---")
print(example_text.output_text)


Retrieved 4 LLM calls from the tape.
From the first retrieved LLM call, the LLM will be trained to predict this text:
---
<|start_header_id|>assistant<|end_header_id|>

Here's a plan to help the user learn about Vulcan Materials using the provided tools:

1. First, use the `get_stock_ticker` function to retrieve the stock ticker symbol for Vulcan Materials by passing "Vulcan Materials" as the `company_name` parameter. This will give us the stock ticker symbol, which we'll use in the next step.

2. Once we have the stock ticker symbol, use the `get_stock_data` function to retrieve the historical stock prices for Vulcan Materials. We can pass the stock ticker symbol, a start date (e.g., 2020-01-01), and an end date (e.g., 2024-09-17) as parameters to get the stock prices for the specified date range.

3. Analyze the retrieved stock prices to provide an overview of Vulcan Materials' stock performance over the specified period. This could include calculating the highest and lowest prices, 

#### Make training data by reusing a tape

Note how `agent2` reuses the tape by `agent1`, even though they have very different prompt and output formats! 

You can inspect the reused tape below and see that the steps are the same as before, but the prompts and outputs are different.

In [18]:
reused_tape, _ = agent2.reuse(final_tape1)
HTML(render_tape_with_prompts(reused_tape, PrettyRenderer()))


We offer a quick way to harness the tape reuse to make training data. 

In [19]:
training_data = agent2.make_training_data(final_tape1)
print(training_data[0].output_text)


<|start_header_id|>assistant<|end_header_id|>

1. Use the `functions.get_stock_ticker` tool to obtain the stock ticker symbol for Vulcan.
2. Use the `functions.get_stock_data` tool to retrieve the stock prices for Vulcan over a specified date range.
3. Summarize the information obtained from the stock ticker and stock data to provide a comprehensive overview of Vulcan's financials.<|eot_id|>


What could be simpler than that?!
We're only scratching the surface of what TapeAgents can do. We invite you to explore the other tutorials and examples to learn more about the TapeAgents framework.

# 5. Multi-agent teams, deeper dive into view stack

Let's add a colleague to our agent that will help it search the internet! 

First thing, we need to give this colleague some tools. In TapeAgents, the agents do not use the environment directly, their interaction with the environment is mediated by in orchestrator such as you application. But the agents should know what tools they can call. And if you are using `main_loop` as the orchestrator, it requires one environment that contains tools of all the agents. 

Let's go ahead and define environments:
- the environments of the root agent
- the one of its internet search specialist colleague
- the environment that contains all the tools.

Note that we won't use the first two environments to produce observations, we'll use them only to generate tool schemas for agent.

In [20]:
from tapeagents.tools.simple_browser import SimpleTextBrowser

browser = SimpleTextBrowser()
search_agent_env = ToolEnvironment([browser.get_search_results, browser.get_page, browser.get_next_page])
# We will use the tool choice mechanism to let the main agent call its search specialist agent.
# To this end, we create a mock tool that represents calling the search agent.
def call_search_agent(query: str):
    """Use this tool to ask a fellow AI agent to search for information on the web."""
    pass
main_agent_env = ToolEnvironment([get_stock_ticker, get_stock_data, call_search_agent])
whole_env = ToolEnvironment([get_stock_ticker, get_stock_data, browser.get_search_results, browser.get_page, browser.get_next_page])

Before we implement the subagent, let's review the way we do multi-agent communication in TapeAgents:
- when the root agent wants to call its subagent "xyz", it puts `Call(agent_name="xyz")` step on the tape
- at the next iteration, the root agent will compute `TapeViewStack` and delegate to the currently active agent. Right after `Call(agent_name="B")` on top of the stack there will be a new view associated with Agent "xyz". The root agent will delegate to "xyz" to make the prompt and to generate the steps from "xyz"'s current node.
- when "xyz" is done it will put `Respond()` step on the tape. At the next iteration the view stack won't have "xyz"'s view on the top any more. 

A reader familiar with the concept of a call stack will find the `TapeViewStack` concept very similar...

Note that for the purpose of computing the view stack the root agent's name is not known. It's view will be signed as "root".

Let's explore the way `Call` and `Respond` works with a minimal example.


In [21]:
from tapeagents.core import Call, Respond

tape = DialogTape(
    steps=[
        UserStep(content="Compute 2 + 2"),
        Call(agent_name="xyz", content="what is 2 + 2, dear xyz"),
        AssistantThought(content="deep thinking by agent xyz"),
        Respond(content="I heard it is 4"),
    ]
)
# We will print a brief summary of view stack after each step
for i in range(0, len(tape)):
    print(f"View stack after step {i}")
    view_stack = TapeViewStack.compute(tape[:i + 1])
    for view in view_stack.stack:
        step_summary = ", ".join([step.__class__.__name__ for step in view.steps])
        print(f"-- {view.agent_full_name}: {step_summary}")
# Note how "root/xyz" view appears after step 1 and disappears after step 3.
# Also note how "root" does not see private thoughts of "xyz" (step 2), and "xyz"
# does not see the initial observation of root (step 0)

View stack after step 0
-- root: UserStep
View stack after step 1
-- root: UserStep, Call
-- root/xyz: Call
View stack after step 2
-- root: UserStep, Call
-- root/xyz: Call, AssistantThought
View stack after step 3
-- root: UserStep, Call, Respond


We are now ready to proceed to the implementation of the internet search agent!

In [22]:

from tapeagents.core import Respond
from tapeagents.prompting import view_to_messages

search_system_instruction = (
    f"""Use at most 5 tool calls to search the request info on on the web."""
)
search_system_message = {"role": "system", "content": search_system_instruction}

class SearchAgentMainNode(Node):
    
    def make_prompt(self, agent, tape: DialogTape) -> Prompt:
        view = agent.compute_view(tape)
        return Prompt(messages=view_to_messages(view.top, agent), tools=search_agent_env.get_tool_schema_dicts())
    
    def generate_steps(self, agent, tape, llm_stream: LLMStream):
        m = llm_stream.get_message()
        if m.content:
            # if the LLM responds, yield Respond(..) as your last step
            yield Respond(content=m.content)
        elif m.tool_calls:
            # while the LLM suggests tool calls, yield them as action steps
            yield ToolCalls(tool_calls=m.tool_calls)
            yield SetNextNode(next_node=0)
        else:
            raise ValueError()
    
search_agent = Agent.create(
    name="search_agent",
    llms=LiteLLM(model_name="gpt-4o", parameters={"temperature": 0.1}),
    nodes=[SearchAgentMainNode()],
)    
# To test the subagent, we'll make a mock root agent that immediately calls "search_agent"
call_step = Call(agent_name="search_agent", content="What influenced Nvidia stock price in late 2022?")
test_root_agent = Agent.create(
    subagents=[search_agent],
    nodes=[Node().with_fixed_steps([call_step])]
)
start_tape = DialogTape()
final_tape = None
for event in main_loop(test_root_agent, start_tape, search_agent_env):
    # We need to stop the loop when "search_agent" responds,
    # otherwise the code will crash because `test_root_agent` only has one node.
    if (ae := event.agent_event) and isinstance(ae.step, Respond):
        final_tape = ae.partial_tape
        break
assert final_tape
HTML(render_tape_with_prompts(final_tape, PrettyRenderer()))

Finally, let's add the search subagent to a financial analyst agent like the ones in earlier examples. We need to give the root agent a way to use the LLM to decide whether to call the search specialist, and if yes, what query to pass to it. We will abuse the tool calling mechanism for this purpose to make the example simpler.

In [23]:
from tapeagents.core import SetNextNode
from tapeagents.dialog_tape import AssistantThought, ToolCalls
from tapeagents.prompting import view_to_messages
from tapeagents.runtime import MainLoopStatus, main_loop
from tapeagents.view import Call
from IPython.display import clear_output

system_instruction = f"""
You will help the user to learn about financials of companies. 
For general user queries, include some info about stock price changes during the last year, as well as some general information on the company.
Today is {today}.
"""
system_message = {"role": "system", "content": system_instruction}

class PlanNode(Node):

    def make_prompt(self, agent, tape) -> Prompt:
        view = agent.compute_view(tape)
        guidance = "Write a natural language plan on how to use tools help the user. Output a list of numbered items, like 1., 2., 3., etc."
        guidance_message = {"role": "user", "content": guidance}
        return Prompt(
            messages=[system_message] + view_to_messages(view.top, agent) + [guidance_message], 
            tools=main_agent_env.get_tool_schema_dicts()
        )

    def generate_steps(self, agent, dialog, llm_stream: LLMStream):
        if content := llm_stream.get_message().content:
            yield AssistantThought(content=content)
        else:
            raise ValueError()

class ActNode(Node):

    def make_prompt(self, agent, tape: DialogTape) -> Prompt:
        view = agent.compute_view(tape)
        guidance = "Follow the plan you created to earlier. When you are done, respond to the user."
        guidance_message = {"role": "user", "content": guidance}
        return Prompt(
            messages=[system_message] + view_to_messages(view.top, agent) + [guidance_message], 
            tools=main_agent_env.get_tool_schema_dicts()
        )

    def generate_steps(self, agent, dialog, llm_stream: LLMStream):
        m = llm_stream.get_message()
        if m.content:
            yield AssistantStep(content=m.content)
            yield SetNextNode(next_node=0)
        elif m.tool_calls:
            yield SetNextNode(next_node=1)
            # only keep the tool calls before the call to another agent
            tool_calls = []
            for tc in m.tool_calls:
                if tc.function.name == "call_search_agent":
                    break
                else:
                    tool_calls.append(tc)
            # either produce the ToolCalls action OR call another agent 
            if tool_calls:
                yield ToolCalls(tool_calls=tool_calls)
            else: 
                tc = m.tool_calls[0]
                assert tc.function.name == "call_search_agent"  
                yield Call(agent_name="search_agent", content=json.loads(m.tool_calls[0].function.arguments)["query"])
            
        else:
            raise ValueError()

multi_agent_analyst = Agent.create(
    name="analyst",
    subagents=[search_agent.clone()],
    llms=LiteLLM(model_name="gpt-4o", parameters={"temperature": 0.1}),
    nodes=[PlanNode(), ActNode()],
)

print("Run the agent!")
start_tape = DialogTape(steps=[UserStep(content="Tell me about Vulcan in 3 sentences")])
for event in main_loop(multi_agent_analyst, start_tape, whole_env):
    # This agent runs for a while, so we will show you a fresh render every time 
    # when the environment finishes reacting with new actions
    if new_tape := event.agent_tape or event.env_tape:
        clear_output()
        display(HTML(render_tape_with_prompts(new_tape, PrettyRenderer())))
    # Uncomment this if you want to pause after every loop
    # if event.env_tape:
    #     input("Press Enter the run the next iteration of the main loop")
    if event.status == MainLoopStatus.EXTERNAL_INPUT_NEEDED:
        break 

Look at this rather long tape and note how "analyst" calls "search_agent", and how the latter then responds to "analyst". Congratulations, you now know how to build a multi-agent TapeAgent!